In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# default_exp eda

In [22]:
import os
import pyspark
from loguru import logger
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行

import seaborn as sns
sns.set(font='Arial Unicode MS')  # 解决Seaborn中文显示问题
import sys
sys.path.append('/Users/luoyonggui/PycharmProjects/mayiutils_n1/mayiutils/data_prepare')
from data_explore import DataExplore as de

# 赛题说明
https://algo.qq.com/index.html?rfisource=DSFISH

本届算法大赛的题目来源于一个重要且有趣的问题。

众所周知，__像用户年龄和性别这样的人口统计学特征是各类推荐系统的重要输入特征__，其中自然也包括了广告平台。

这背后的假设是，__用户对广告的偏好会随着其年龄和性别的不同而有所区别__。许多行业的实践者已经多次验证了这一假设。

然而，大多数验证所采用的方式都是以人口统计学属性作为输入来产生推荐结果，然后离线或者在线地对比用与不用这些输入的情况下的推荐性能。

本届大赛的题目尝试从另一个方向来验证这个假设，即__以用户在广告系统中的交互行为作为输入来预测用户的人口统计学属性__。


我们认为这一赛题的“逆向思考”本身具有其研究价值和趣味性，此外也有实用价值和挑战性。例如，对于缺乏用户信息的实践者来说，基于其自有系统的数据来推断用户属性，可以帮助其在更广的人群上实现智能定向或者受众保护。与此同时，参赛者需要综合运用机器学习领域的各种技术来实现更准确的预估。

具体而言，在比赛期间，我们将为参赛者提供一组用户在长度为91天（3个月）的时间窗口内的广告点击历史记录作为训练数据集。每条记录中包含了日期（从1到91）、用户信息（年龄，性别），被点击的广告的信息（素材id、广告id、产品id、产品类目id、广告主id、广告主行业id等），以及该用户当天点击该广告的次数。测试数据集将会是另一组用户的广告点击历史记录。提供给参赛者的测试数据集中不会包含这些用户的年龄和性别信息。

本赛题要求参赛者预测测试数据集中出现的用户的年龄和性别，并以约定的格式提交预测结果。大赛官网后台将会自动计算得分并排名。详情参见【评估方式】和【提交方式】。

初赛和复赛除了所提供的训练数据集的量级有所不同之外，其他设置均相同

## 赛程
即日起-5.31 报名

5.7-6.22 初赛

6.23-7.22 复赛

8月 决赛答辩

## 数据说明
训练数据中包含了一组用户91天的广告点击日志，被组织为三张表，以带标题行的CSV文件的格式提供（编码采用无BOM的UTF-8），分别是：click_log.csv，user.csv，ad.csv。测试数据包含了另一组用户91天的广告点击日志，组织方式与训练数据相同，但不包含user.csv。各表的详细格式描述如下（字段顺序以下面的描述为准，各字段用逗号’,’分隔，中间无空格）：

### click_log.csv:
* time: 天粒度的时间，整数值，取值范围[1,91]。
* user_id:从1到N随机编号生成的不重复的加密的用户id，其中N为用户总数目（训练集和测试集）。
* creative_id:用户点击的广告素材的id，采用类似于user_id的方式生成。
* click_times: 当天该用户点击该广告素材的次数

### user.csv:
* user_id
* age: 分段表示的用户年龄，取值范围[1-10]。
* gender：用户性别，取值范围[1,2]

### ad.csv:
* creative_id
* ad_id:该素材所归属的广告的id，采用类似于user_id的方式生成。每个广告可能包含多个可展示的素材。
* product_id:该广告中所宣传的产品的id，采用类似于user_id的方式生成。
* product_category: 该广告中所宣传的产品的类别id，采用类似于user_id的方式生成。
* advertiser_id: 广告主的id，采用类似于user_id的方式生成。
* industry:广告主所属行业的id，采用类似于user_id的方式生成

## 提交方式
参赛者提交的结果为一个带标题行的submission.csv文件，编码采用无BOM的UTF-8，具体格式如下（字段顺序以下面的描述为准，各字段用逗号分隔，中间无空格）：
* user_id
* predicted_age: 预测的用户年龄分段，取值范围[1,10]。
* predicted_gender: 预测的用户性别，取值范围[1,2]。

测试数据集中每个用户均应在submission.csv文件中对应有且仅有一行预测结果。各用户的预测结果在该文件中的出现顺序与评估结果无关。

## 评估方式
大赛会根据参赛者提交的结果计算预测的准确率（accuracy）。

年龄预测和性别预测将分别评估准确率，两者之和将被用作参赛者的打分。

测试数据集会和训练数据集一起提供给参赛者。

大赛会将测试数据集中出现的用户划分为两组，具体的划分方式对参赛者不可见。

其中一组用户将被用于初赛和复赛阶段除最后一天之外的排行榜打分计算，另一组则用于初赛和复赛阶段最后一天的排行榜打分计算，以及最后的胜出队伍选择。

# 智能钛机器学习平台
https://cloud.tencent.com/document/product/851

https://cloud.tencent.com/document/product/851/39086

智能钛机器学习平台是为 AI 工程师打造的一站式机器学习服务平台，为用户提供从数据预处理、模型构建、模型训练、模型评估到模型服务的全流程开发及部署支持。智能钛机器学习平台内置丰富的算法组件，支持多种算法框架，满足多种 AI 应用场景的需求。自动化建模（AutoML）的支持与拖拽式任务流设计让 AI 初学者也能轻松上手。

## 创建 COS 存储桶

# 解题思路
## mine
很直观的是一个20分类问题
## 鱼佬
https://zhuanlan.zhihu.com/p/130913652

# EDA

In [4]:
test_dir = 'data_origin/test'
train_dir = 'data_origin/train_preliminary/'

In [8]:
train_click_log = pd.read_csv(os.path.join(train_dir, 'click_log.csv'))
train_user = pd.read_csv(os.path.join(train_dir, 'user.csv'))
train_ad = pd.read_csv(os.path.join(train_dir, 'ad.csv'))

In [15]:
test_click_log = pd.read_csv(os.path.join(test_dir, 'click_log.csv'))
test_ad = pd.read_csv(os.path.join(test_dir, 'ad.csv'))

In [18]:
train_user['label'] = train_user['age'].map(str) + '_' + train_user['gender'].map(str)

In [9]:
train_click_log.head(2)

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1


In [19]:
train_user.head(2)

,user_id,age,gender,label
0,1,4,1,4_1
1,2,10,1,10_1


In [10]:
train_ad.head(2)

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202


## click_log

In [12]:
de.describe(train_click_log)

num of records: 30082771, num of columns: 4


,Data Type,Unique Values,count Missing,% Missing,Mode,Count Mode,% Mode,mean,std,min,25%,50%,75%,max
time,int64,91,0,0.0,85,417797,1.38882,4.801738e+01,2.592504e+01,1.0,26.0,49.0,71.0,91.0
user_id,int64,900000,0,0.0,839368,16868,0.056072,4.498556e+05,2.599939e+05,1.0,224613.0,449804.0,675242.0,900000.0
creative_id,int64,2481135,0,0.0,108133,103033,0.342498,1.676532e+06,1.308054e+06,1.0,415810.0,1508864.0,2740463.5,4445718.0
click_times,int64,41,0,0.0,1,28426543,94.4944,1.063127e+00,2.982831e-01,1.0,1.0,1.0,1.0,152.0


In [16]:
de.describe(test_click_log)

num of records: 33585512, num of columns: 4


,Data Type,Unique Values,count Missing,% Missing,Mode,Count Mode,% Mode,mean,std,min,25%,50%,75%,max
time,int64,91,0,0.0,85,467552,1.39212,4.797323e+01,2.593779e+01,1.0,26.0,49.0,71.0,91.0
user_id,int64,1000000,0,0.0,3648518,113974,0.339355,3.499714e+06,2.883313e+05,3000001.0,3249486.0,3500907.0,3748320.0,4000000.0
creative_id,int64,2618159,0,0.0,108133,113726,0.338616,1.674632e+06,1.308090e+06,1.0,415805.0,1506880.0,2737762.0,4445720.0
click_times,int64,93,0,0.0,1,31695705,94.3731,1.066196e+00,3.561287e-01,1.0,1.0,1.0,1.0,185.0


## ad

In [14]:
de.describe(train_ad)

num of records: 2481135, num of columns: 6


,Data Type,Unique Values,count Missing,% Missing,Mode,Count Mode,% Mode,mean,std,min,25%,50%,75%,max
creative_id,int64,2481135,0,0.0,1,1,4.03041e-05,2.19201e+06,1.2855e+06,1,1.07637e+06,2.19195e+06,3.30442e+06,4.44572e+06
ad_id,int64,2264190,0,0.0,1.72996e+06,19,0.000765779,1.88811e+06,1.09976e+06,1,939192,1.88902e+06,2.83946e+06,3.8122e+06
product_id,object,33273,0,0.0,\N,929524,37.4637,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_category,int64,18,0,0.0,2,923994,37.2408,7.80593,6.77056,1,2,5,18,18
advertiser_id,int64,52090,0,0.0,14681,26906,1.08442,27416.2,15346.3,2,14974,25197,38254,62965
industry,object,326,0,0.0,247,262634,10.5852,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
de.describe(test_ad)

num of records: 2618159, num of columns: 6


,Data Type,Unique Values,count Missing,% Missing,Mode,Count Mode,% Mode,mean,std,min,25%,50%,75%,max
creative_id,int64,2618159,0,0.0,1,1,3.81948e-05,2.19213e+06,1.28501e+06,1,1.0775e+06,2.18861e+06,3.30423e+06,4.44572e+06
ad_id,int64,2379475,0,0.0,681564,22,0.000840285,1.88815e+06,1.09937e+06,1,940042,1.88622e+06,2.83931e+06,3.8122e+06
product_id,object,34111,0,0.0,\N,974696,37.2283,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_category,int64,18,0,0.0,2,974647,37.2264,7.81163,6.76541,1,2,5,18,18
advertiser_id,int64,52861,0,0.0,14681,27386,1.046,27553,15360.2,1,15137,25305,38416,62965
industry,object,326,0,0.0,247,275504,10.5228,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## user

In [20]:
de.describe(train_user)

num of records: 900000, num of columns: 4


,Data Type,Unique Values,count Missing,% Missing,Mode,Count Mode,% Mode,mean,std,min,25%,50%,75%,max
user_id,int64,900000,0,0.0,1,1,0.000111111,450000,259808,1,225001,450000,675000,900000
age,int64,10,0,0.0,3,202909,22.5454,4.2461,2.00355,1,3,4,6,10
gender,int64,2,0,0.0,1,602610,66.9567,1.33043,0.470369,1,1,1,2,2
label,object,20,0,0.0,3_1,134473,14.9414,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# 数据挺干净的，每天的用户点击没有重复的数据
train_click_log.groupby(['time', 'user_id','creative_id'])['time'].count().sort_values()

time  user_id  creative_id
1     12       214927         1
64    23836    1544479        1
               1207205        1
               671429         1
               66299          1
                             ..
34    376108   1455785        1
               1414499        1
      376107   621969         1
      376099   1253109        1
91    900000   4387250        1
Name: time, Length: 30082771, dtype: int64